## 概覽

```UpstageDocumentParseLoader``` 是 Upstage 所設計的一款強大文件分析工具，可作為 LangChain 框架中的文件載入器使用。其專長在於透過分析文件版面與內容，將其轉換為結構化的 HTML 格式。

**主要特色**：

- **完整的版面分析**：  
  能分析並辨識多種文件格式（如 PDF、圖片）中的結構元素，例如標題、段落、表格與圖片。

- **自動化結構辨識**：  
  根據閱讀順序自動偵測並序列化文件中的元素，準確轉換為 HTML 結構。

- **可選的 OCR 支援**：  
  提供光學文字辨識（OCR）功能，處理掃描或圖片型文件。OCR 模式包含：
  
  - ```force```：強制使用 OCR 從圖片中擷取文字。
  - ```auto```：自動從 PDF 中擷取文字（若輸入非 PDF 將拋出錯誤）。

透過辨識並保留文件元素間的邏輯關係，```UpstageDocumentParseLoader``` 能實現精準且具上下文意識的文件分析。

**從 Layout Analysis 遷移說明**：  
Upstage 推出了 Document Parse，以取代原有的 Layout Analysis！新版本支援更多文件格式，並具備 markdown 輸出、圖表偵測、數學公式辨識等新功能，更多特性也在持續開發中。舊版 Layout Analysis（layout-analysis-0.4.0）將於 2024 年 11 月 10 日正式終止維護。

### Table of Contents 

- [Overview](#overview)
- [Key Changes from Layout Analysis](#key-changes-from-layout-analysis)
- [Environment Setup](#environment-setup)
- [UpstageDocumentParseLoader Key Parameters](#upstagedocumentparseloader-key-parameters)
- [Usage Example](#usage-example)

### 與 Layout Analysis 的主要差異（Key Changes from Layout Analysis）

**現有選項的變更**：

1. ```use_ocr``` → ```ocr```  
   原本的 ```use_ocr``` 選項已被 ```ocr``` 取代。  
   原先使用布林值（```True/False```），現在改為接受更具控制性的字串值：  
   - ```force```：強制進行 OCR  
   - ```auto```：自動辨識 PDF 並執行 OCR（非 PDF 將報錯）

2. ```output_type``` → ```output_format```  
   原先用於指定輸出格式的 ```output_type``` 已重新命名為 ```output_format```，語意更清晰。

3. ```exclude``` → ```base64_encoding```  
   用來排除特定元素的 ```exclude``` 選項，已被 ```base64_encoding``` 所取代。  
   現在可以指定是否將某些類別的元素（如圖像）以 Base64 編碼格式保留輸出。

### References
- [UpstageDocumentParseLoader](https://python.langchain.com/api_reference/upstage/document_parse/langchain_upstage.document_parse.UpstageDocumentParseLoader.html)
- [UpstageLayoutAnalysisLoader](https://python.langchain.com/api_reference/upstage/layout_analysis/langchain_upstage.layout_analysis.UpstageLayoutAnalysisLoader.html)
- [Upstage Migrate to Document Parse from Layout Analysis](https://console.upstage.ai/docs/capabilities/document-parse/migration-dp)

----

## Environment Setup
Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]** 

- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials.
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.


### API Key Configuration
To use ```UpstageDocumentParseLoader``` , you need to [obtain a Upstage API key](https://console.upstage.ai/api-keys).

Once you have your API key, set it as the value for the variable ```UPSTAGE_API_KEY``` .


In [73]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [74]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain_upstage",
    ],
    verbose=False,
    upgrade=False,
)

In [75]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "UPSTAGE_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "12-UpstageDocumentParseLoader",
    }
)

Environment variables have been set successfully.


You can alternatively set ```UPSTAGE_API_KEY``` in .env file and load it.

[Note] This is not necessary if you've already set ```UPSTAGE_API_KEY``` in previous steps.

In [76]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [77]:
import os
import nest_asyncio

# Allow async
nest_asyncio.apply()

## UpstageDocumentParseLoader 主要參數說明

- ```file_path```：  
  欲分析的文件路徑，可為單一檔案或多個檔案。

- ```split```：  
  文件切分模式  
  可選值：`'none'`（不切分）、`'element'`（依元素切分）、`'page'`（依頁面切分）  
  預設值：`'none'`

- ```model```：  
  用於文件解析的模型名稱  
  預設值：`'document-parse'`

- ```ocr```：  
  OCR 模式  
  可選值：  
  - `"force"`：一律使用 OCR 進行文字擷取  
  - `"auto"`：僅在輸入為 PDF 時啟用 OCR

- ```output_format```：  
  分析結果的輸出格式  
  可選值：`'html'`、`'text'`、`'markdown'`  
  預設值：`'html'`

- ```coordinates```：  
  是否在輸出中包含 OCR 區塊的座標位置  
  預設值：`True`

- ```base64_encoding```：  
  指定哪些元素類別需以 Base64 編碼輸出  
  可接受的類別包括：  
  `'paragraph'`、`'table'`、`'figure'`、`'header'`、`'footer'`、`'list'`、`'chart'` 等

## 使用範例（Usage Example）

現在讓我們透過一個實際範例，來執行 ```UpstageDocumentParseLoader```。

### 資料準備

在本教學中，我們將使用以下 PDF 檔案進行文件分析：

- 下載連結：  
  [Modular-RAG: Transforming RAG Systems into LEGO-like Reconfigurable Frameworks](https://arxiv.org/abs/2407.21059)

- 檔案名稱：`2407.21059.pdf`  
- 檔案路徑：`./data/2407.21059.pdf`

請從上述連結下載該 PDF 文件，並在你目前的工作目錄下建立一個名為 `data` 的資料夾，將檔案儲存至該資料夾中。

In [78]:
# Download and save sample PDF file to ./data directory
import requests


def download_pdf(url, save_path):
    """
    Downloads a PDF file from the given URL and saves it to the specified path.

    Args:
        url (str): The URL of the PDF file to download.
        save_path (str): The full path (including file name) where the file will be saved.
    """
    try:
        # Ensure the directory exists
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        # Download the file
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an error for bad status codes

        # Save the file to the specified path
        with open(save_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        print(f"PDF downloaded and saved to: {save_path}")
    except Exception as e:
        print(f"An error occurred while downloading the file: {e}")


# Configuration for the PDF file
pdf_url = "https://arxiv.org/pdf/2407.21059"
file_path = "./data/2407.21059.pdf"

# Download the PDF
download_pdf(pdf_url, file_path)

PDF downloaded and saved to: ./data/2407.21059.pdf


In [79]:
# Set file path
FILE_PATH = "data/2407.21059.pdf"  # modify to your file path

In [80]:
from langchain_upstage import UpstageDocumentParseLoader

# Configure the document loader
loader = UpstageDocumentParseLoader(
    FILE_PATH,
    output_format="html",
    split="page",
    ocr="auto",
    coordinates=True,
    base64_encoding=["chart"],
)

# Load the document
docs = loader.load()

# Print the results
for doc in docs[:2]:
    print(doc)

page_content='<p id='0' data-category='paragraph' style='font-size:14px'>1</p> <h1 id='1' style='font-size:20px'>Modular RAG: Transforming RAG Systems into<br>LEGO-like Reconfigurable Frameworks</h1> <br><p id='2' data-category='paragraph' style='font-size:18px'>Yunfan Gao, Yun Xiong, Meng Wang, Haofen Wang</p> <p id='3' data-category='paragraph' style='font-size:16px'>Abstract—Retrieval-augmented Generation (RAG) has<br>markedly enhanced the capabilities of Large Language Models<br>(LLMs) in tackling knowledge-intensive tasks. The increasing<br>demands of application scenarios have driven the evolution<br>of RAG, leading to the integration of advanced retrievers,<br>LLMs and other complementary technologies, which in turn<br>has amplified the intricacy of RAG systems. However, the rapid<br>advancements are outpacing the foundational RAG paradigm,<br>with many methods struggling to be unified under the process<br>of “retrieve-then-generate”. In this context, this paper examines<br>the 